In [0]:
# if gsync_save:
#     try:
#         import utils
#     except ModuleNotFoundError:
#         !wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
#         import utils

#     gsync = utils.GDriveSync()
# try:
#     from allconv import AllConvNet
# except ModuleNotFoundError: 
#     !wget https://github.com/StefOe/all-conv-pytorch/raw/HEAD/allconv.py
#     from allconv import AllConvNet

In [0]:
cuda = True
train_batch_size = 32
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, transforms

In [0]:

import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class AllConvNet(nn.Module):
    def __init__(self, input_size, n_classes=10, **kwargs):
        super(AllConvNet, self).__init__()
        self.conv1 = nn.Conv2d(input_size, 96, 5, padding=1)
        self.conv2 = nn.Conv2d(96, 96, 1, padding=1)
        #self.maxPool3=nn.MaxPool2d(3,stride=2,padding=1)
        self.maxPool3=nn.MaxPool2d(3,stride=2)
        
        self.conv4 = nn.Conv2d(96, 192, 5, padding=1)
        self.conv5 = nn.Conv2d(192, 192, 1, padding=1)
        #self.maxPool6=nn.MaxPool2d(3,stride=2,padding=1)
        self.maxPool6=nn.MaxPool2d(3,stride=2)
        
        self.conv7 = nn.Conv2d(192, 192, 3, padding=1)
        self.conv8 = nn.Conv2d(192, 192, 1)
        self.class_conv = nn.Conv2d(192, n_classes, 1)
    
    def forward(self, x):
        x_drop = F.dropout(x, .2)
        conv1_out = F.relu(self.conv1(x_drop))
        conv2_out = F.relu(self.conv2(conv1_out))
        maxPool3_out = F.relu(self.maxPool3(conv2_out))
        maxPool3_out_drop = F.dropout(maxPool3_out, .5)
        conv4_out = F.relu(self.conv4(maxPool3_out_drop))
        conv5_out = F.relu(self.conv5(conv4_out))
        maxPool6_out = F.relu(self.maxPool6(conv5_out))
        maxPool6_out_drop = F.dropout(maxPool6_out, .5)
        conv7_out = F.relu(self.conv7(maxPool6_out_drop))
        conv8_out = F.relu(self.conv8(conv7_out))

        class_out = F.relu(self.class_conv(conv8_out))
        pool_out = F.adaptive_avg_pool2d(class_out, 1)
        pool_out.squeeze_(-1)
        pool_out.squeeze_(-1)
        # pool_out_softmax=F.softmax(pool_out,dim=1)
        # return pool_out_softmax
        return pool_out

trial=AllConvNet(3)
#print(trial)
summary(trial.cuda(), (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 30, 30]           7,296
            Conv2d-2           [-1, 96, 32, 32]           9,312
         MaxPool2d-3           [-1, 96, 15, 15]               0
            Conv2d-4          [-1, 192, 13, 13]         460,992
            Conv2d-5          [-1, 192, 15, 15]          37,056
         MaxPool2d-6            [-1, 192, 7, 7]               0
            Conv2d-7            [-1, 192, 7, 7]         331,968
            Conv2d-8            [-1, 192, 7, 7]          37,056
            Conv2d-9             [-1, 10, 7, 7]           1,930
Total params: 885,610
Trainable params: 885,610
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 2.37
Params size (MB): 3.38
Estimated Total Size (MB): 5.76
-------------------------------------------

In [0]:
cuda =torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = trainset.data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)

print(train_mean)
print(train_std)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
[0.49139968 0.48215841 0.44653091]
[0.24703223 0.24348513 0.26158784]


In [0]:
model = AllConvNet(3)
if cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[200, 250, 300], gamma=0.1)

In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(epoch, best_loss, best_epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(model, "best.pt")
        # if gsync_save:
        #     gsync.update_file_to_folder("best.pt")
    return best_loss, best_epoch

In [0]:
for epoch in range(350):
    scheduler.step()
    train(epoch)
    best_loss, best_epoch = test(epoch, best_loss, best_epoch)

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.306128
Train Epoch: 0 [3200/50000 (6%)]	Loss: 2.302307
Train Epoch: 0 [6400/50000 (13%)]	Loss: 2.288123
Train Epoch: 0 [9600/50000 (19%)]	Loss: 2.307241
Train Epoch: 0 [12800/50000 (26%)]	Loss: 2.297220
Train Epoch: 0 [16000/50000 (32%)]	Loss: 2.305354
Train Epoch: 0 [19200/50000 (38%)]	Loss: 2.289232
Train Epoch: 0 [22400/50000 (45%)]	Loss: 2.328951
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.259106
Train Epoch: 0 [28800/50000 (58%)]	Loss: 2.281955
Train Epoch: 0 [32000/50000 (64%)]	Loss: 2.197603
Train Epoch: 0 [35200/50000 (70%)]	Loss: 2.263151
Train Epoch: 0 [38400/50000 (77%)]	Loss: 2.253476
Train Epoch: 0 [41600/50000 (83%)]	Loss: 2.203083
Train Epoch: 0 [44800/50000 (90%)]	Loss: 2.247094
Train Epoch: 0 [48000/50000 (96%)]	Loss: 2.157040

Test set: Average loss: 0.0178, Accuracy: 1820/10000 (18%)



/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type AllConvNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Streaming output truncated to the last 5000 lines.
Test set: Average loss: 0.0057, Accuracy: 7630/10000 (76%)

Train Epoch: 87 [0/50000 (0%)]	Loss: 0.549475
Train Epoch: 87 [3200/50000 (6%)]	Loss: 0.516751
Train Epoch: 87 [6400/50000 (13%)]	Loss: 0.515275
Train Epoch: 87 [9600/50000 (19%)]	Loss: 0.711593
Train Epoch: 87 [12800/50000 (26%)]	Loss: 0.734434
Train Epoch: 87 [16000/50000 (32%)]	Loss: 0.625560
Train Epoch: 87 [19200/50000 (38%)]	Loss: 0.715049
Train Epoch: 87 [22400/50000 (45%)]	Loss: 0.462746
Train Epoch: 87 [25600/50000 (51%)]	Loss: 0.589769
Train Epoch: 87 [28800/50000 (58%)]	Loss: 0.573494
Train Epoch: 87 [32000/50000 (64%)]	Loss: 0.523033
Train Epoch: 87 [35200/50000 (70%)]	Loss: 0.412174
Train Epoch: 87 [38400/50000 (77%)]	Loss: 0.716566
Train Epoch: 87 [41600/50000 (83%)]	Loss: 0.356317
Train Epoch: 87 [44800/50000 (90%)]	Loss: 0.478758
Train Epoch: 87 [48000/50000 (96%)]	Loss: 0.622121

Test set: Average loss: 0.0056, Accuracy: 7644/10000 (76%)

Train Epoch: 88 [0/50

In [0]:
for epoch in range(350):
    train(epoch)
    scheduler.step()
    best_loss, best_epoch = test(epoch, best_loss, best_epoch)

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.302559
Train Epoch: 0 [3200/50000 (6%)]	Loss: 2.302578
Train Epoch: 0 [6400/50000 (13%)]	Loss: 2.302579
Train Epoch: 0 [9600/50000 (19%)]	Loss: 2.302583
Train Epoch: 0 [12800/50000 (26%)]	Loss: 2.302571
Train Epoch: 0 [16000/50000 (32%)]	Loss: 2.302574
Train Epoch: 0 [19200/50000 (38%)]	Loss: 2.302572
Train Epoch: 0 [22400/50000 (45%)]	Loss: 2.302585
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.302585
Train Epoch: 0 [28800/50000 (58%)]	Loss: 2.302577
Train Epoch: 0 [32000/50000 (64%)]	Loss: 2.302594
Train Epoch: 0 [35200/50000 (70%)]	Loss: 2.302552
Train Epoch: 0 [38400/50000 (77%)]	Loss: 2.302584
Train Epoch: 0 [41600/50000 (83%)]	Loss: 2.302561
Train Epoch: 0 [44800/50000 (90%)]	Loss: 2.302575
Train Epoch: 0 [48000/50000 (96%)]	Loss: 2.302572

Test set: Average loss: 0.0187, Accuracy: 1445/10000 (14%)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.302579
Train Epoch: 1 [3200/50000 (6%)]	Loss: 2.302569
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.302556
Train 

Exception ignored in: <bound method Image.__del__ of <PIL.Image.Image image mode=RGB size=32x32 at 0x7EFF0F79BDA0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/PIL/Image.py", line 630, in __del__
    self.__exit__()
  File "/usr/local/lib/python3.6/dist-packages/PIL/Image.py", line 592, in __exit__
    if hasattr(self, "fp") and getattr(self, "_exclusive_fp", False):
KeyboardInterrupt


KeyboardInterrupt: ignored

In [0]:
for epoch in range(350):
    scheduler.step()
    train(epoch)
    best_loss, best_epoch = test(epoch, best_loss, best_epoch)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.302683
Train Epoch: 0 [3200/50000 (6%)]	Loss: 2.302541
Train Epoch: 0 [6400/50000 (13%)]	Loss: 2.302517
Train Epoch: 0 [9600/50000 (19%)]	Loss: 2.303138
Train Epoch: 0 [12800/50000 (26%)]	Loss: 2.302233
Train Epoch: 0 [16000/50000 (32%)]	Loss: 2.302501
Train Epoch: 0 [19200/50000 (38%)]	Loss: 2.302290
Train Epoch: 0 [22400/50000 (45%)]	Loss: 2.302685
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.302814
Train Epoch: 0 [28800/50000 (58%)]	Loss: 2.302715
Train Epoch: 0 [32000/50000 (64%)]	Loss: 2.302251
Train Epoch: 0 [35200/50000 (70%)]	Loss: 2.302529
Train Epoch: 0 [38400/50000 (77%)]	Loss: 2.302695
Train Epoch: 0 [41600/50000 (83%)]	Loss: 2.302758
Train Epoch: 0 [44800/50000 (90%)]	Loss: 2.302735
Train Epoch: 0 [48000/50000 (96%)]	Loss: 2.303213

Test set: Average loss: 0.0187, Accuracy: 1000/10000 (10%)



/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type AllConvNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Streaming output truncated to the last 5000 lines.
Test set: Average loss: 0.0164, Accuracy: 4334/10000 (43%)

Train Epoch: 87 [0/50000 (0%)]	Loss: 1.983411
Train Epoch: 87 [3200/50000 (6%)]	Loss: 1.879928
Train Epoch: 87 [6400/50000 (13%)]	Loss: 2.077430
Train Epoch: 87 [9600/50000 (19%)]	Loss: 2.081548
Train Epoch: 87 [12800/50000 (26%)]	Loss: 2.060868
Train Epoch: 87 [16000/50000 (32%)]	Loss: 1.984022
Train Epoch: 87 [19200/50000 (38%)]	Loss: 2.087568
Train Epoch: 87 [22400/50000 (45%)]	Loss: 1.956902
Train Epoch: 87 [25600/50000 (51%)]	Loss: 2.091907
Train Epoch: 87 [28800/50000 (58%)]	Loss: 1.991959
Train Epoch: 87 [32000/50000 (64%)]	Loss: 1.976677
Train Epoch: 87 [35200/50000 (70%)]	Loss: 1.994844
Train Epoch: 87 [38400/50000 (77%)]	Loss: 1.898705
Train Epoch: 87 [41600/50000 (83%)]	Loss: 1.995006
Train Epoch: 87 [44800/50000 (90%)]	Loss: 2.100032
Train Epoch: 87 [48000/50000 (96%)]	Loss: 1.985940

Test set: Average loss: 0.0163, Accuracy: 4433/10000 (44%)

Train Epoch: 88 [0/50

In [0]:
for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        print(target)
        break

tensor([8, 4, 5, 8, 4, 9, 5, 4, 5, 6, 5, 6, 3, 5, 8, 0, 8, 9, 9, 9, 8, 3, 5, 8,
        5, 4, 6, 1, 3, 6, 7, 7], device='cuda:0')
